In [13]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [246]:
in_dir = "/Users/hn/Desktop/Desktop/Kirti/check_point/analogs/"
out_dir = "/Users/hn/Desktop/"

In [244]:
all_data_usa = pd.read_csv(in_dir + "all_data_usa.csv") 
avg_rcp45 = pd.read_csv(in_dir + "averaged_data_rcp45.csv") 

In [245]:
all_data_usa.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,1979,32.46875_-109.90625,68,2.939262,1.319697,7.080458,1.348167,0.0,300.525,4821.668922,0,observed
1,1979,32.46875_-109.96875,68,2.946050,1.245847,7.104994,1.396980,0.0,303.925,4831.770062,0,observed


In [88]:
avg_rcp45.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,2026,43.53125_-116.59375,92,2.352305,23.368053,27.85959,3.009202,0.025247,268.491667,3669.118839,1,ensembe_mean
1,2026,43.59375_-116.78125,92,2.336201,24.035184,27.50817,2.912981,0.027137,260.750000,3623.723188,1,ensembe_mean


## Append

  - First append the needed treated unit to all historical/control units so we can find nearest locations.
  - Then take numeric part of the data to be able to operate on them.
        - normalize
        - find distances
  - add the distances back to the data frame.
  - sort the data frame according to distances.

In [208]:
one_site = avg_rcp45.iloc[0, :].copy()
dt = all_data_usa.append(one_site)

#### Take the numerical values so we can work with them, normalize them, compute KNNs

In [209]:
dt_numeric = dt.drop(['year', 'location', 'treatment', 'ClimateScenario'], axis=1).copy()

In [210]:
dt_numeric.tail(2)

,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd
47840,119,2.060168,81.778861,45.945466,0.552418,0.000000,286.150000,2970.529512
0,92,2.352305,23.368053,27.859590,3.009202,0.025247,268.491667,3669.118839


### Normalize data

In [211]:
x = dt_numeric.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_dt = pd.DataFrame(x_scaled)

In [219]:
normalized_dt.tail(2)

,0,1,2,3,4,5,6,7
47840,0.636943,0.386943,0.931307,0.700336,0.026941,0.000000,0.095527,0.299025
47841,0.464968,0.479293,0.266118,0.424657,0.146756,0.006785,0.088829,0.424869


### compute Euclidean distances 
Subtract the last row from all other rows to compute distances.

In [213]:
subtracted = normalized_dt - normalized_dt.iloc[-1, ]

In [215]:
subtracted.head(2)

,0,1,2,3,4,5,6,7
0,-0.152866,0.185546,-0.251089,-0.316732,-0.081007,-0.006785,0.012149,0.207621
1,-0.152866,0.187692,-0.251930,-0.316358,-0.078627,-0.006785,0.013439,0.209440


In [216]:
distances = np.linalg.norm(subtracted, axis=1, keepdims=True)

In [217]:
dt['distances'] = distances
dt = dt.sort_values(by=['distances'])

In [218]:
dt.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario,distances
0,2026,43.53125_-116.59375,92,2.352305,23.368053,27.859590,3.009202,0.025247,268.491667,3669.118839,1,ensembe_mean,0.000000
37049,2007,43.53125_-116.78125,96,2.344302,26.196502,31.237424,2.176265,0.000000,185.725000,3513.329901,0,observed,0.088392
20240,1994,43.53125_-116.78125,99,2.284805,25.017968,27.828322,1.544439,0.000000,220.825000,3547.658606,0,observed,0.093547
37055,2007,43.59375_-116.84375,96,2.327640,26.807480,31.149851,2.006078,0.000000,181.925000,3490.173473,0,observed,0.096518
17659,1992,43.59375_-116.78125,91,2.331342,20.596046,31.195072,1.578679,0.000000,188.325000,3526.934183,0,observed,0.100795


In [232]:
out_name = one_site['location'] + '_' + str(one_site['year']) + ".csv"
out_name

'43.53125_-116.59375_2026.csv'

In [247]:
dt.to_csv(out_dir + out_name, sep='\t')